


# Análise exploratória de dados do Titanic

### Resumo
---
 
 [Introdução](#Introdução)

 [Conjunto de dados](#Conjunto-de-dados)
 
 [Ferramentas](#Ferramentas)

 [Questões-chave](#Questões-chave)

 Considerações

 Divulgação

 [Referências](#Referências)

 ### Introdução
 ...

### Ferramentas

Para análise foi utilizada a linguagem Python e suas bibliotecas: **pandas** para estrutura e manipulação de dados; **numpy** para operação com matrizes e vertores; **matplotlib** e **seaborn** para visualização de dados; **funções personalizadas** para manipulação dos dados; **formatação de tabela** personalizada para aprimorar a exibição dos dados.

In [4]:
# bibliotecas usadas para análise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

# bibliotecas usadas para análise
from data.data_modules import data_manipulation_functions as dm
from data.data_modules import data_table_format as tab

### Conjunto de dados

#### Fonte
A análise utilizou como material os dados fornecidos pela Udacity a partir do Kaggle e representa 891 das 2.224 pessoas que estavam a bordo do Titanic.  
Link para a fonte dos dados https://www.kaggle.com/c/titanic/data

#### Preparação dos dados
Para deixar os dados mais legíveis e consistentes de modo a preservar a informação, os dados passaram por um processo de limpeza. Durante a execução algumas colunas foram avaliadas com relação à qualidade e pertinência de serem utilizadas na análise. Assim, dentre as 12 colunas foram delimitadas as seguintes 10 colunas:
> **survived** (informa se sobreviveu), **ticket_class** (classe), **name** (nome), **gender** (sexo), **age** (idade), **siblings_or_spouses** (quantidade de irmãs(ãos) ou  cônjuges), **parents_or_children** (quantidade de pais ou filhas(os)), **ticket_number** (número do bilhete), **fare** (custo) e **embarkation_city** (cidade de embarque)

Dentre as colunas, a que contém os dados referentes a idade possuia cerca de 80% dos dados completos. Para permitir cruzamentos entre outras variáveis, os cerca de 20% de dados foram preenchidos de modo randômico de modo a preservar a proporção de diferençãs de idade dos dados completos.

O passo a passo realizado pode ser verificado em [/data/data_wrangling.ipynb](./data/data_wrangling.ipynb)

In [12]:
# carregamento dos dados
data_titanic = pd.read_csv('./data/data_titanic_clean')

# exibição das 3 primeiras linhas do conjunto de dados
tab.format_table(data_titanic.head(10))

,name,age,gender,ticket_class,fare,ticket_number,embarkation_city,siblings_or_spouses,parents_or_children,survived
0,"Braund, Mr. Owen Harris",22,male,third,7.25,A/5 21171,Southampton,1,0,no
1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",38,female,first,71.2833,PC 17599,Cherbourg,1,0,yes
2,"Heikkinen, Miss. Laina",26,female,third,7.925,STON/O2. 3101282,Southampton,0,0,yes
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35,female,first,53.1,113803,Southampton,1,0,yes
4,"Allen, Mr. William Henry",35,male,third,8.05,373450,Southampton,0,0,no
5,"Moran, Mr. James",20.9093,male,third,8.4583,330877,Queestown,0,0,no
6,"McCarthy, Mr. Timothy J",54,male,first,51.8625,17463,Southampton,0,0,no
7,"Palsson, Master. Gosta Leonard",2,male,third,21.075,349909,Southampton,3,1,no
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",27,female,third,11.1333,347742,Southampton,0,2,yes
9,"Nasser, Mrs. Nicholas (Adele Achem)",14,female,second,30.0708,237736,Cherbourg,1,0,yes


#### Descrição da amostra
A amostra contém 891 dados que representa cerca de 20% da quantidade total de 2224 referentes às pessoas presentes no navio. 

Por meio dela não é possível distinguir entre pessoas passageiras e tripulação. Por meio dos
Possui diferença de gênero x x x, idade x x x mas não é possível confirmar se ela representa proporcionalmente a quantidade total.
Além disso, considera-se que os dados se referem a pessoas passageiras e porque não há informações sobre pessoas da tripulação.

NameError: name 'data_titanic_raw' is not defined

### Questões-chave
A presente análise investiga temas relacionados a idade, genêro e econômico a partir da amostra de dados do Titanic. 
Para chegar às questões da análise, primeiramente foram elaboradas algumas antes da limpeza dos dados e outras depois.
Assim, as seguintes questões-chave foram elencadas:

#### Questões iniciais
- Houve diferença representativa na proporção de pessoas sobreviventes de acordo com o preço pago na passagem?
- É possível confirmar que a crença de que mulheres e crianças foram prioridade no salvamento?

#### Questões a partir da limpeza dos dados
- Houve diferenças relevantes na distribuição das classes por local de embarque?
- Pessoas que viajaram sozinhas tiveram mais changes de sobreviver?

**Critérios**:
- "Faça testes e sugira perguntas que promovam olhar para relacionamentos entre múltiplas variáveis"
- "Você deve analisar pelo menos uma variável dependente e três variáveis ​​independentes em sua investigação"
- "Uma definição de qual pergunta você fez"
- "Uma nota especificando qual foi o conjunto de dados usado"
- "Uma descrição do que você fez para investigar a pergunta"
Um PDF ou HTML contendo sua análise. Este arquivo deve conter:


Descrição de qualquer limpeza de dados feita  
Um resumo das estatísticas e gráficos comunicando seu resultado final  
O código usado para criar suas análises. Se você usou um notebook iPython, pode enviar seu .ipynb. Caso não o tenha feito, encaminhe os arquivos .py.



Para obter maior compreensão dos dados, segue uma descrição... ?? 

Cabeçalhos mais legíveis
remoção da coluna a passengerId não agregava a análise
ajustes nas colunas , , , para permitir verificar valores de modo mais claro
É possível verificar o passo a passo aqui

### Referências

##### Fonte
- [Kaggle - Titanic](https://www.kaggle.com/c/titanic/data)

##### Conteúdos sobre Titanic:
- [Em naufrágios, crianças e mulheres têm mesmo prioridade?](https://super.abril.com.br/mundo-estranho/em-naufragios-criancas-e-mulheres-tem-mesmo-prioridade/)
- [Em naufrágios e outros desastres, mulheres e crianças só conseguem sair por último](http://revistagalileu.globo.com/Revista/Common/0,,EMI302308-17770,00-EM+NAUFRAGIOS+E+OUTROS+DESASTRES+MULHERES+E+CRIANCAS+SO+CONSEGUEM+SAIR+POR+.html)
- [Nerdologia - Titanic](https://www.youtube.com/watch?v=GlEV_2qbvx8)


##### Análises de dados
- [Lautert, Larissa. Análise exploratória da fila - hacking-health-sc-2017](https://github.com/llautert/hacking-health-sc-2017/blob/master/An%C3%A1lise%20explorat%C3%B3ria%20da%20fila.ipynb)
- [Reis, Filipe - T2 - Análise Exploratória de Dados - Filipe Reis - RA091202](https://github.com/fireis/ia369w_exp_data_ana/blob/master/T2%20-%20Saude%20-%20Filipe%20Reis%20-%20RA091202.ipynb)
- [Siqueira, Fernando - Análise de notas de alunos](https://github.com/fernandolkf/dotdigitalgroup/blob/master/dotdigitalgroup.ipynb)


##### Técnicos
- [Handling missing data](https://www.oreilly.com/learning/handling-missing-data)
- [Mean Median Mode lines showing only in last graph in seaborn](https://stackoverflow.com/questions/51417483/mean-median-mode-lines-showing-only-in-last-graph-in-seaborn/51417635)

- [A collection of various notebook extensions for Jupyter](https://github.com/ipython-contrib/jupyter_contrib_nbextensions)
- [Python for Data Science and Machine Learning Bootcamp](https://www.udemy.com/python-for-data-science-and-machine-learning-bootcamp)